In [ ]:
import pandas as pd
df_part = pd.read_csv(r"F:\celeba\archive\list_eval_partition.csv")

train = df_part[df_part['partition'] == 0]['image_id'].tolist()
val = df_part[df_part['partition'] == 1]['image_id'].tolist()
test = df_part[df_part['partition'] == 2]['image_id'].tolist()

In [ ]:
df = pd.read_csv(r"F:\celeba\archive\list_attr_celeba.csv")
df_train = df[df['image_id'].isin(train)]
df_val = df[df['image_id'].isin(val)]
df_test = df[df['image_id'].isin(test)]

In [ ]:
df.columns 

In [ ]:

attrs = ["Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "High_Cheekbones", "Mouth_Slightly_Open", "Chubby", "Eyeglasses", "Gray_Hair", "Narrow_Eyes", "Smiling", "Wearing_Hat"]

df.loc[0, attrs].values

In [ ]:
import torch
# (df.loc[0, attrs].values + 1) // 2
torch.as_tensor((df.loc[0, attrs].values + 1) // 2)

In [ ]:
import main
from tqdm import tqdm
from torch.utils.data  import DataLoader
import torch

test_dataloader = DataLoader(main.test_ds, batch_size=256, num_workers=3, pin_memory=True, persistent_workers=True)
model = main.CelebAAttributeModel.load_from_checkpoint(r"F:\celeba\lightning_logs\version_13\checkpoints\epoch=61-step=39431.ckpt").cuda()


In [ ]:
all_labels = []
all_preds = []
for image, labels in tqdm(test_dataloader):
    with torch.no_grad():
        out = model(image.cuda())
        all_preds.append(out)
        all_labels.append(labels)


In [26]:
labels = torch.cat(all_labels).cuda()
preds = torch.cat([torch.cat(x, 1) for x in all_preds])
labels.shape, preds.shape

(torch.Size([19962, 13]), torch.Size([19962, 13]))

In [29]:
preds_binary = (preds >= 0.5)
accuracies = [(labels[:, i] == preds_binary[:, i]).float().mean() for i in range(13)]
accuracies, ["Bags_Under_Eyes", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair", "High_Cheekbones", "Mouth_Slightly_Open", "Chubby", "Eyeglasses", "Gray_Hair", "Narrow_Eyes", "Smiling", "Wearing_Hat"]

([tensor(0.8276, device='cuda:0'),
  tensor(0.9556, device='cuda:0'),
  tensor(0.8759, device='cuda:0'),
  tensor(0.9507, device='cuda:0'),
  tensor(0.8632, device='cuda:0'),
  tensor(0.8614, device='cuda:0'),
  tensor(0.9341, device='cuda:0'),
  tensor(0.9532, device='cuda:0'),
  tensor(0.9965, device='cuda:0'),
  tensor(0.9800, device='cuda:0'),
  tensor(0.8661, device='cuda:0'),
  tensor(0.9253, device='cuda:0'),
  tensor(0.9907, device='cuda:0')],
 ['Bags_Under_Eyes',
  'Bangs',
  'Black_Hair',
  'Blond_Hair',
  'Brown_Hair',
  'High_Cheekbones',
  'Mouth_Slightly_Open',
  'Chubby',
  'Eyeglasses',
  'Gray_Hair',
  'Narrow_Eyes',
  'Smiling',
  'Wearing_Hat'])